# Laboratorio 4. Series de tiempo
El objetivo de este laboratorio es poner en práctica el análisis de series de tiempo usando Python

## Actividad 1. Lectura de datos

In [16]:
# Conectando con Drive
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#!pip install pandas

In [18]:
import pandas as pd

#df_pib = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/API_PER_DS2_es_excel_v2_2764705.xls',
#                       sheet_name='PIB per cápita', header=2, index_col=0)
df_pib = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_peru.csv',
                     sep=";", index_col=0)
df_pib.index = df_pib.index.astype('str') + "-12-31"
df_pib.index = pd.to_datetime(df_pib.index)
display(df_pib)
print(df_pib.dtypes)
print(df_pib.describe())


UnicodeDecodeError: ignored

## Actividad 2. Visualización preliminar


In [ ]:
import matplotlib.pyplot as plt

#print(df_pib)

plt.figure(figsize=(12,8))
plt.plot(df_pib)
plt.xlabel('Años')
plt.ylabel('PIB per cápita (US$ a precios actuales)')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib.png')
plt.show()

## Actividad 2.1. Descripción

In [ ]:
from pandas.plotting import lag_plot

#print(df_pib.describe())

plt.figure(figsize=(12,8))
plt.hist(df_pib.PIB, density=True)
df_pib.PIB.plot(kind='kde')
plt.xlabel('PIB per cápita (US$ a precios actuales)')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_hist.png')
plt.show()

plt.figure(figsize=(12,8))
lag_plot(df_pib.PIB, lag=1)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_lag1.png')
plt.show()


## Actividad 3. Media móvil

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(df_pib)
plt.plot(df_pib.rolling(window = 7, center=True).mean())
plt.xlabel('Años')
plt.ylabel('PIB per cápita (US$ a precios actuales)')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.legend(["Serie", "MA(7)"], fontsize=20)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_ma.png')
plt.show()

## Actividad 4. Función de autocorrelación

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
from pandas.plotting import autocorrelation_plot

alpha = 0.1
fig, ax = plt.subplots(figsize=(12,8))
fig = plot_acf(df_pib.PIB, ax=ax, alpha=alpha, lags=50, title=None)
plt.xlabel('Retraso')
plt.ylabel('Autocorrelación')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.annotate("IC del " + str(100*(1-alpha)) + "% para la nulidad de la autocorrelación", xy=(30,0.8))
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_autocorr.png')
plt.show()

plt.figure(figsize=(12,8))
autocorrelation_plot(df_pib.PIB)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf

alpha = 0.05
fig, ax = plt.subplots(figsize=(12,8))
fig = plot_pacf(df_pib.PIB, ax=ax, alpha=alpha, lags=20, title=None)
plt.xlabel('Retraso')
plt.ylabel('Autocorrelación parcial')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.annotate("IC del " + str(100*(1-alpha)) + "% para la nulidad de la autocorrelación parcial", xy=(10,0.5))
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_pautocorr.png')
plt.show()

## Variograma


In [ ]:
from statsmodels.tsa.stattools import acf

df_acf = acf(df_pib.PIB, fft=False, nlags=50)
print(df_acf)
print(len(df_acf))
l = []
G = []
asin = []
for i in range(1,50):
  l.append(i)
  G.append((1-df_acf[i])/(1-df_acf[1]))
  asin.append(1/(1-df_acf[1]))

plt.figure(figsize=(12,8))
plt.scatter(x=l, y=G)
plt.plot(asin, color="brown")
plt.xlabel('Retrasos')
plt.ylabel('Variograma (Gk)')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.annotate(r"Asíntota en $1/(1-\rho_1)$", xy=(30,22))
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_var.png')
plt.show()

## Actividad 5. Transformación de Box-Cox


In [ ]:
from scipy.stats import boxcox

ar_bc, l = boxcox(df_pib.PIB)
df_pib['PIB_BC'] = ar_bc
display(df_pib)

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB_BC)
plt.xlabel('Años')
plt.ylabel(r"Serie transformada con $\lambda=$"+str(round(l,4)))
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_bc.png')
plt.show()

plt.figure(figsize=(12,8))
plt.hist(df_pib.PIB_BC, density=True)
df_pib.PIB_BC.plot(kind='kde')
plt.xlabel('PIB per cápita (US$ a precios actuales)')
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_hist.png')
plt.show()


## Prueba de Dickey-Fuller

$H_0$: La serie de tiempo es no-estacionaria

In [ ]:
from statsmodels.tsa.stattools import adfuller

resulta = adfuller(df_pib.PIB)
print('ADF Statistic: %f' % resulta[0])
print('p-value: %f' % resulta[1])
print('Critical Values:')
for key, value in resulta[4].items():
  print('\t%s: %.3f' % (key, value))

## Actividad 6. Diferenciando la serie

In [ ]:
df_pib['PIB_dif1'] = df_pib.PIB.diff()
df_pib['PIB_dif2'] = df_pib.PIB.diff(2)
display(df_pib)

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB)
plt.plot(df_pib.PIB_dif1)
plt.plot(df_pib.PIB_dif2)
plt.xlabel('Años')
plt.ylabel("PIB per cápita (US$ a precios actuales)")
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.legend(["Serie original", "Serie diferenciada una vez", "Serie diferenciada dos veces"], fontsize=14)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_dif.png')
plt.show()

resulta = adfuller(df_pib.PIB_dif2[3:], )
print('ADF Statistic: %f' % resulta[0])
print('p-value: %f' % resulta[1])
print('Critical Values:')
for key, value in resulta[4].items():
  print('\t%s: %.3f' % (key, value))

In [ ]:
df_pib['PIB_BC_dif1'] = df_pib.PIB_BC.diff()

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB_BC_dif1)
plt.xlabel('Años')
plt.ylabel("PIB per cápita (US$ a precios actuales) Transformado (B-C)")
plt.grid(axis='y', linestyle='dashdot', linewidth=1)
plt.legend(["Serie transformada y diferenciada una vez"], fontsize=14)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_bs_dif.png')
plt.show()

resulta = adfuller(df_pib.PIB_BC_dif1[2:], )
print('ADF Statistic: %f' % resulta[0])
print('p-value: %f' % resulta[1])
print('Critical Values:')
for key, value in resulta[4].items():
  print('\t%s: %.3f' % (key, value))

## Descomposición clásica de una serie


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Suponiendo que la estacionalidad se produce por trienios

descom = seasonal_decompose(df_pib['PIB'], model='multiplicative', period=3)
print("Tendencia:\n", descom.trend)
print("Estacionalidad:\n", descom.seasonal)
print("Error:\n", descom.resid)

plt.figure(figsize=(10,8))
descom.plot()
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_descom.png')
plt.show()


## Suavizamiento exponencial
Método de Holt...

In [ ]:
from statsmodels.tsa.api import Holt

holt = Holt(df_pib.PIB, exponential=True)
holtfit = holt.fit()

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB, color='black', label="Datos originales")
plt.plot(holtfit.fittedvalues, color='blue', label="Datos suavizados")
plt.xlabel('Años', fontsize=14)
plt.ylabel("PIB per cápita (US$ a precios actuales)", fontsize=14)
plt.legend(fontsize=14)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_holt.png')
plt.show

## Modelos ARIMA


In [ ]:
!pip install pmdarima

In [ ]:
df_pib

In [ ]:
from pmdarima.arima import auto_arima
import numpy as np

mod = auto_arima(df_pib.PIB, max_p=10, max_q=10)
print(mod)
print(mod.summary())
df_pib['fitted'] = mod.predict_in_sample(df_pib.PIB)

n_per = 10
pred, ci = mod.predict(n_periods=n_per, return_conf_int=True)

df_pred = pd.DataFrame({'PIB_Pred': pred, 'PIB_Pred_li': ci[:,0],
                        'PIB_Pred_ls': ci[:,1]})
print(df_pred)
ultimo_ind = df_pib.shape[0] - 1
ultimo_año = df_pib.iloc[[ultimo_ind]].index 
seq = range(ultimo_año.year[0]+1, ultimo_año.year[0]+1+n_per, 1)
df_pred['fecha'] = seq
df_pred.fecha = df_pred.fecha.astype('str') + "-12-31"
df_pred.index = pd.to_datetime(df_pred.fecha)
df_pred = df_pred.drop('fecha', axis=1)
print(df_pred)

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB, color='black', label="Datos originales")
plt.plot(df_pib.fitted, color='blue', label="Datos ajustados")
plt.plot(df_pred.PIB_Pred, color='green', label="Pronósticos")
plt.plot(df_pred.PIB_Pred_li, color='green', linestyle='dashed', label="Li(95%) Pronósticos")
plt.plot(df_pred.PIB_Pred_ls, color='green', linestyle='dashed', label="Ls(95%) Pronósticos")
plt.xlabel('Años', fontsize=14)
plt.ylabel("PIB per cápita (US$ a precios actuales)", fontsize=14)
plt.legend(fontsize=14)
plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_pred.png')
plt.show()

df_pib['res'] = mod.resid()
res = pd.DataFrame(mod.resid())
plt.figure(figsize=(12, 8))
df_pib.res.plot()
plt.show()
plt.figure(figsize=(12, 8))
df_pib.res.plot(kind='kde')
plt.show()

## Ejercicio

Investigar el funcionamiento de $\texttt{ARIMA}$ en $\texttt{statsmodels.tsa.arima.model}$. Rehacer este problema. ¿Será ARIMA(7,2,6) un mejor modelo?

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

mod = ARIMA(df_pib.PIB, order=(7, 2, 6))
res = mod.fit()
print(res.summary())

df_pib['fitted2'] = res.fittedvalues

n_per = 10
pred = res.forecast(n_per)

df_pred = pd.DataFrame({'PIB_Pred': pred})
print(df_pred)
ultimo_ind = df_pib.shape[0] - 1
ultimo_año = df_pib.iloc[[ultimo_ind]].index 
seq = range(ultimo_año.year[0]+1, ultimo_año.year[0]+1+n_per, 1)
df_pred['fecha'] = seq
df_pred.fecha = df_pred.fecha.astype('str') + "-12-31"
df_pred.index = pd.to_datetime(df_pred.fecha)
df_pred = df_pred.drop('fecha', axis=1)
print(df_pred)

plt.figure(figsize=(12, 8))
plt.plot(df_pib.PIB, color='black', label="Datos originales")
plt.plot(df_pib.fitted2, color='blue', label="Datos ajustados")
plt.plot(df_pred.PIB_Pred, color='green', label="Pronósticos")
#plt.plot(df_pred.PIB_Pred_li, color='green', linestyle='dashed', label="Li(95%) Pronósticos")
#plt.plot(df_pred.PIB_Pred_ls, color='green', linestyle='dashed', label="Ls(95%) Pronósticos")
plt.xlabel('Años', fontsize=14)
plt.ylabel("PIB per cápita (US$ a precios actuales)", fontsize=14)
plt.legend(fontsize=14)
#plt.savefig('/content/drive/MyDrive/Colab Notebooks/DSRP/Modelos estadísticos/pib_pred.png')
plt.show()

## Tarea
¿Cómo sería un esquema de machine learning para seleccionar el mejor modelo de entre algunos candidatos?